Coursera capstone

In [1]:
zomatao_user_key = 'b348f057e5edb73209c2caa7f6dcb638'
kol_geojson = 'https://raw.githubusercontent.com/datameet/Municipal_Spatial_Data/master/Kolkata/kolkata.geojson'

In [2]:
# Zomato web acesss
class Zomato:
    def __init__(self, user_key, content_type = 'application/json'):
        self._key = user_key
        self.request_count = 0
        self.content_type = content_type
        self._header = {
                    'Accept': content_type,
                    'user-key': user_key,
                            }
        self._categories_id = {}

    def get_response(self,api,params=''):
        import requests
        headers = self._header
        status = {  200 :'Sucessfully Connected to Zomato..',
                    403 :'Connection Error: Invalid User Keys',
                    404 : 'No Connection to Server..'
                }
        response = requests.get(api, headers=headers,params=params)
        print(status[response.status_code])
        if response.status_code == 200:
            self.request_count += 1
            return response.json()
    
    def categories(self,method='get'):
        api = 'https://developers.zomato.com/api/v2.1/categories'
        response = self.get_response(api)
        result_dict = {categories['categories']['name']: categories['categories']['id'] for categories in response['categories']}
        if method == 'set':
            self._categories_id = result_dict 
        return result_dict
    
    def cities(self,city_name='',id='',lat='',lon=''):
        params = (
            ('q', city_name),
            ('lat', lat),
            ('lon', lon),
            ('city_ids', id))

        api = 'https://developers.zomato.com/api/v2.1/cities'
        response = self.get_response(api,params=params)  
        return response

    def cuisines(self,city_id='',lat='',lon=''):
        params = (
            ('city_id', city_id),
            ('lat', lat),
            ('lon', lon))
        api = 'https://developers.zomato.com/api/v2.1/cuisines'
        response = self.get_response(api,params=params)  
        return response
    
    def geocode(self,lat='',lon=''):
        api = 'https://developers.zomato.com/api/v2.1/geocode'
        params = (
            ('lat', lat),
            ('lon', lon))
        response = self.get_response(api,params=params)  
        return response

    def location(self,city='',lat='',lon='',max_count=100):
        api = 'https://developers.zomato.com/api/v2.1/locations'
        params = (
        ('query', city),
        ('lat', lat),
        ('lon', lon),
        ('count', str(max_count)))
        response = self.get_response(api,params=params)  
        return response
    
    def location_details(self,city='',lat='',lon=''):
        r = self.location(city,lat,lon,1)
        entity_id = r['location_suggestions'][0]['entity_id']
        entity_type = r['location_suggestions'][0]['entity_type']

        params = (
            ('entity_id', entity_id),
            ('entity_type', entity_type))
        api = 'https://developers.zomato.com/api/v2.1/location_details'

        response = self.get_response(api,params=params)  
        return response
    
    def restaurant(self,res_id=''):
        api = 'https://developers.zomato.com/api/v2.1/restaurant'
        params = (
                ('res_id', res_id),)
        response = self.get_response(api,params=params) 
        return response
    
    def reviews(self,res_id='',start='1',count='10'):
        api = 'https://developers.zomato.com/api/v2.1/reviews'
        params = (
            ('res_id', res_id),
            ('start', start),
            ('count', count),) 
        response = self.get_response(api,params=params) 
        return response


In [3]:
zomato = Zomato(zomatao_user_key)

In [10]:
def geo(pincode,latitude,longitude):
    geo_dict = {
        'pincode':pincode, 'latitude': latitude, 'longitude': longitude
    }
    geo_result = zomato.geocode(latitude,longitude)
    geo_dict['location'] = geo_result['location']['title']
    geo_dict['subzone'] = geo_result['popularity']['subzone']
    geo_dict['top_cuisines'] = ','.join(geo_result['popularity']['top_cuisines'])
    geo_dict['popularity'] = geo_result['popularity']['popularity']
    geo_dict['nearby_res'] = ','.join(geo_result['popularity']['nearby_res'])

    return geo_dict

pd.DataFrame(geo('700063','22.464592','88.310792'),index=[0])
geo('700063','22.464592','88.310792')

Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..


{'pincode': '700063',
 'latitude': '22.464592',
 'longitude': '88.310792',
 'location': 'Behala',
 'subzone': 'Thakur Pukur',
 'top_cuisines': 'North Indian,Chinese,Biryani,Fast Food,Bengali',
 'popularity': '3.52',
 'nearby_res': '23408,18420484,18240469,18309975,22424,18473784,18662812,18572855,25351'}

In [4]:
# kolkata pincode corrdinates
from bs4 import BeautifulSoup #Web Scrapping
import pandas as pd #library for dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from tabulate import tabulate
import json #library for json file
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

with open("pincode.html", "r") as f:
    
    contents = f.read()

    soup = BeautifulSoup(contents, 'lxml')

pincodes = soup.find_all('div',class_='jbzYp')
pincodes = [pc.string for pc in pincodes]

cordinates = soup.find_all('div',class_='title')
cordinates = [(c.string.split(", ")[0][:7],c.string.split(", ")[1][:7]) for c in cordinates]

kolkata_pincodes = pd.DataFrame({'pincode':pincodes,'cordinates':cordinates}).sort_values(by='pincode')
#kolkata_pincodes.to_csv("kolkata.csv")

In [15]:
kolkata = pd.read_csv('kolkata_resturent.csv',index_col=0)
kolkata.head()

,pincode,latitude,longitude,location,subzone,top_cuisines,popularity,nearby_res
114,700122,22.766851,88.376411,Barrackpore,Barrackpore,"Chinese,North Indian,Fast Food,Bengali,Mughlai",4.17,"17983797,18311142,18698012,23973,18663023,2364..."
113,700121,22.766614,88.422983,Barrackpore,Barrackpore,"Chinese,North Indian,Fast Food,Bengali,Mughlai",4.17,"17983797,18311142,18698012,23973,18663023,2364..."
112,700120,22.752200,88.379000,Barrackpore,Barrackpore,"Chinese,North Indian,Fast Food,Bengali,Mughlai",4.17,"17983797,18311142,18698012,23973,18663023,2364..."
111,700119,22.734883,88.396685,Khardah,Khardah,"Chinese,North Indian,Fast Food,Biryani,Bengali",3.95,"19127565,24311,25341,19114341,18198698,1860263..."
109,700117,22.723511,88.375425,Khardah,Khardah,"Chinese,North Indian,Fast Food,Biryani,Bengali",3.95,"19127565,24311,25341,19114341,18198698,1860263..."


In [147]:
address = 'Kolkata, IN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} City are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Kolkata, IN City are 22.54541245, 88.3567751581234.


In [17]:
map_kolkata = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, pincode in zip(kolkata['latitude'], kolkata['longitude'],kolkata['pincode']):
    label = '{}'.format(pincode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kolkata)  
    
map_kolkata

In [70]:
def add_marker(dataframe,map_,color='blue'):
    for lat, lng, pincode,location in zip(dataframe['latitude'], dataframe['longitude'],dataframe['pincode'],dataframe['subzone']):
        label = '{}({})'.format(location,pincode)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_)
    return map_

In [89]:
kolkata_grby_subzone = kolkata.groupby('subzone').max().reset_index()
kolkata_grby_subzone.head()

,subzone,pincode,latitude,longitude,location,top_cuisines,popularity,nearby_res
0,Ajoy Nagar,700099,22.492526,88.403415,Ajoy Nagar,"Chinese,North Indian,Fast Food,Bengali,Beverages",4.37,"18607141,20965,18265446,18997514,21859,1840420..."
1,Alipore,700027,22.526000,88.334100,Alipore,"Chinese,North Indian,Fast Food,Bakery,Beverages",4.30,"18343756,18476619,18354589,21461,20168,24428,1..."
2,Baguihati,700059,22.613800,88.433900,Baguihati,"Chinese,North Indian,Fast Food,Bengali,Biryani",4.57,"24485,18848647,22165,18676394,18166925,1836165..."
3,Ballygunge,700042,22.531333,88.366080,Ballygunge,"Chinese,Fast Food,North Indian,Beverages,Desserts",4.95,"19088819,19236255,18707652,19213376,18017612,1..."
4,Bangur,700055,22.611100,88.409100,Bangur,"Chinese,North Indian,Fast Food,Beverages,Desserts",4.86,"19059355,18568762,19300247,23227,21483,1816692..."


In [91]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_kolkata_ = folium.Map(location=[latitude, longitude], zoom_start=10)

#label 
kolkata_4 = kolkata_grby_subzone[kolkata_grby_subzone.popularity > 4.0]
kolkata_3to4 = kolkata_grby_subzone[kolkata_grby_subzone.popularity.between(3.0, 3.99, inclusive=True)]
kolkata_2to3 = kolkata_grby_subzone[kolkata_grby_subzone.popularity.between(2.0, 2.99, inclusive=True)]
kolkata_1to2 = kolkata_grby_subzone[kolkata_grby_subzone.popularity.between(1.0, 1.99, inclusive=True)]
kolkata_1 = kolkata_grby_subzone[kolkata_grby_subzone.popularity < 1.0]

# add markers to map
add_marker(kolkata_4,map_kolkata_,color='#381460')
add_marker(kolkata_3to4,map_kolkata_,color='#b21f66')
add_marker(kolkata_2to3,map_kolkata_,color='#fe346e')
add_marker(kolkata_1,map_kolkata_,color='#ffbd69')
add_marker(kolkata_1to2,map_kolkata_,color='green')
map_kolkata_

In [ ]:
import json
for res in pin_63['nearby_restaurants']:
    print()
    print(res['restaurant']['name']," : ", res['restaurant']['id'])
    print(res['restaurant']['location']['address'])
    print(res['restaurant']['cuisines'])
    print("{},{}".format(res['restaurant']['user_rating']['rating_text'],res['restaurant']['user_rating']['aggregate_rating']))

In [123]:
def resturent(res_id=''):
    res = zomato.restaurant(res_id)
    res_dict = {}
    res_dict['name'] = res['name']
    res_dict['id'] = res['id']
    res_dict['address'] = res['location']['address']
    res_dict['zipcode'] = res['location']['zipcode']
    res_dict['locality'] = res['location']['locality']
    res_dict['latitude'] = res['location']['latitude']
    res_dict['longitude'] = res['location']['longitude']
    res_dict['cuisines'] = res['cuisines']
    res_dict['average_cost_for_two'] = res['average_cost_for_two']
    res_dict['highlights'] = ','.join(res['highlights'])
    res_dict['aggregate_rating'] = res['user_rating']['aggregate_rating']
    res_dict['number_of_reviews'] = res['all_reviews_count']
    return res_dict

pd.DataFrame(resturent('17983797'),index=[0])

Sucessfully Connected to Zomato..


,name,id,address,zipcode,locality,latitude,longitude,cuisines,average_cost_for_two,highlights,aggregate_rating,number_of_reviews
0,Dada Boudi Restaurant,17983797,"12/10, S.N. Bannerjee Road, Barrackpore, Kolkata",,Barrackpore,22.7591812398,88.3702220768,"Biryani, North Indian, Chinese, Mughlai",600,"Dinner,Cash,Takeaway Available,Lunch,Delivery,...",4.4,1116


In [119]:
res_id_list = []
for nrby_res in kolkata_grby_subzone['nearby_res']:
    res_id_list.extend(nrby_res.split(','))

res_id_list = list(set(res_id_list))

In [125]:
import time
resturent_df ={}
for i in range(len(res_id_list)):
    resturent_df[i] = resturent(res_id_list[i])
    time.sleep(2) 

Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Connected to Zomato..
Sucessfully Co

In [130]:
resturent_details = pd.DataFrame(resturent_df).T
resturent_details.head()

,name,id,address,zipcode,locality,latitude,longitude,cuisines,average_cost_for_two,highlights,aggregate_rating,number_of_reviews
0,Fiesta,24590,"438, Santoshpur Avenue, Santoshpur, Kolkata",700075,Santoshpur,22.4934368272,88.3891655132,"Chinese, Thai",700,"Dinner,Delivery,Credit Card,Lunch,Cash,Takeawa...",4.0,259
1,Food Junction,18577170,"167, Shreerampur Garia, Kanungo Park, Garia, K...",,Garia,22.4655656134,88.3792198822,"Biryani, North Indian, Chinese",400,"Dinner,Cash,Takeaway Available,Debit Card,Lunc...",3.9,395
2,Rang De Basanti Dhaba,19206717,"29F, 4th Floor, Traffic More, BT Road, North B...",,Sodepur,22.7015228769,88.3747010306,North Indian,800,"Takeaway Available,Debit Card,Dinner,Cash,Cred...",3.9,130
3,Sengupta's,23571,"1/14, Poddar Nagar, Near Girls High School, Pr...",700068,Prince Anwar Shah Road,22.5013038181,88.3638464659,"Bengali, Biryani, North Indian, Chinese",300,"Dinner,Cash,Takeaway Available,Lunch,Delivery,...",3.5,1406
4,Adil Biryani Centre,22833,"38, Shamsul Huda Road, Ballygunge, Kolkata",,Ballygunge,22.5371674983,88.3700909838,"Biryani, Mughlai",150,"Lunch,Cash,Takeaway Available,Dinner,Delivery,...",3.9,153


In [134]:
resturent_details.to_csv('resturent_details.csv')

In [142]:
resturent_details.aggregate_rating = pd.to_numeric(resturent_details.aggregate_rating)
resturent_details.dtypes

name                     object
id                       object
address                  object
zipcode                  object
locality                 object
latitude                 object
longitude                object
cuisines                 object
average_cost_for_two     object
highlights               object
aggregate_rating        float64
number_of_reviews        object
dtype: object

## Import _Foursquare_ Credential

In [149]:
CLIENT_ID = 'UBLX5C1RQM4EQJIT20SYTFY2YAMDC51IIGFTHGEHKXR2T44B' # Foursquare ID
CLIENT_SECRET = 'OLN5KSDUUAEMCYYCQTC35GDTBV13FDHEYDZS5YER3SLPLMVB' #Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: UBLX5C1RQM4EQJIT20SYTFY2YAMDC51IIGFTHGEHKXR2T44B
CLIENT_SECRET:OLN5KSDUUAEMCYYCQTC35GDTBV13FDHEYDZS5YER3SLPLMVB


In [152]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


#Extract Venues from FourSquare as lat and long given as input
def foursquareVenue(neighborhood_name, neighborhood_latitude, neighborhood_longitude,redius=500,limit=100):
    
    """Extract Venues from FourSquare as lat and long given as input"""
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
     CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
    
    print("Neighborhood Name: {}".format(neighborhood_name))

    #Get Url resopnse from FourSquare
    from pandas.io.json import json_normalize
    import requests
    response = json_normalize(requests.get(url).json()["response"]['groups'][0]['items']) #Save as json file as normalized datafarme
    # clean columns
    columns_name = ['vanue','vanue_category','venue_latitude','venue_longitude']
    if response.empty:
        venues = pd.DataFrame(np.nan,index=[0],columns= columns_name)
    else:
    # filter columns
        venues = response.loc[:, ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']]

        # filter the category for each row
        # use function get_category_type() apply row wise in category object
        venues['venue.categories'] = venues.apply(get_category_type, axis=1)
        # clean columns
        venues.columns = columns_name
    #Add Neighborhood Details
    venues['neighborhood_name'], venues['neighborhood_latitude'], venues['neighborhood_longitude'] = neighborhood_name, neighborhood_latitude, neighborhood_longitude

    return venues


In [153]:
foursquareVenue('700063',float('22.464592'),float('88.310792'))

Neighborhood Name: 700063


,vanue,vanue_category,venue_latitude,venue_longitude,neighborhood_name,neighborhood_latitude,neighborhood_longitude
0,pora aswathotalla,Bus Station,22.468529,88.309008,700063,22.464592,88.310792


In [157]:
column_names = foursquareVenue('700063',float('22.464592'),float('88.310792')).columns
foursquare_resturent = pd.DataFrame(columns = column_names)

Neighborhood Name: 700063


In [159]:
kolkata_grby_subzone.head()

,subzone,pincode,latitude,longitude,location,top_cuisines,popularity,nearby_res
0,Ajoy Nagar,700099,22.492526,88.403415,Ajoy Nagar,"Chinese,North Indian,Fast Food,Bengali,Beverages",4.37,"18607141,20965,18265446,18997514,21859,1840420..."
1,Alipore,700027,22.526000,88.334100,Alipore,"Chinese,North Indian,Fast Food,Bakery,Beverages",4.30,"18343756,18476619,18354589,21461,20168,24428,1..."
2,Baguihati,700059,22.613800,88.433900,Baguihati,"Chinese,North Indian,Fast Food,Bengali,Biryani",4.57,"24485,18848647,22165,18676394,18166925,1836165..."
3,Ballygunge,700042,22.531333,88.366080,Ballygunge,"Chinese,Fast Food,North Indian,Beverages,Desserts",4.95,"19088819,19236255,18707652,19213376,18017612,1..."
4,Bangur,700055,22.611100,88.409100,Bangur,"Chinese,North Indian,Fast Food,Beverages,Desserts",4.86,"19059355,18568762,19300247,23227,21483,1816692..."


In [162]:
for subzone, latitude, longitude in zip(kolkata_grby_subzone['subzone'],kolkata_grby_subzone['latitude'],kolkata_grby_subzone['longitude']):
    x = foursquareVenue(subzone,  float(latitude), float(longitude))
    foursquare_resturent = pd.concat([foursquare_resturent,x],axis=0)

Neighborhood Name: Ajoy Nagar
Neighborhood Name: Alipore
Neighborhood Name: Baguihati
Neighborhood Name: Ballygunge
Neighborhood Name: Bangur
Neighborhood Name: Baranagar
Neighborhood Name: Barasat
Neighborhood Name: Barrackpore
Neighborhood Name: Behala
Neighborhood Name: Belghoria
Neighborhood Name: Bhawanipur
Neighborhood Name: Bow Bazar
Neighborhood Name: Chinar Park
Neighborhood Name: Chowringhee
Neighborhood Name: College Street
Neighborhood Name: Dalhousie BBD Bagh
Neighborhood Name: Desapriya Park
Neighborhood Name: Dhakuria
Neighborhood Name: Dum Dum
Neighborhood Name: Elgin
Neighborhood Name: Entally
Neighborhood Name: Garia
Neighborhood Name: Golf Green
Neighborhood Name: Hatibagan
Neighborhood Name: Jadavpur
Neighborhood Name: Joka
Neighborhood Name: Kaikhali
Neighborhood Name: Kankurgachi
Neighborhood Name: Kasba
Neighborhood Name: Kestopur
Neighborhood Name: Khardah
Neighborhood Name: Kidderpore
Neighborhood Name: Kona Exp Way
Neighborhood Name: Lake Town
Neighborhood Nam

In [163]:
foursquare_resturent.head()

,vanue,vanue_category,venue_latitude,venue_longitude,neighborhood_name,neighborhood_latitude,neighborhood_longitude
0,Cafe Coffee Day,Coffee Shop,22.493967,88.400788,Ajoy Nagar,22.492526,88.403415
1,Bikrampur Medical Hall,Pharmacy,22.490961,88.403300,Ajoy Nagar,22.492526,88.403415
2,The Sugarr & Spice,Bakery,22.490921,88.404305,Ajoy Nagar,22.492526,88.403415
3,Samrat Guest House,Hotel,22.490734,88.403114,Ajoy Nagar,22.492526,88.403415
4,HDFC Bank ATM,ATM,22.491672,88.399969,Ajoy Nagar,22.492526,88.403415


In [164]:
foursquare_resturent.shape

(345, 7)

In [166]:
foursquare_resturent.isnull().sum()

vanue                     16
vanue_category            16
venue_latitude            16
venue_longitude           16
neighborhood_name          0
neighborhood_latitude      0
neighborhood_longitude     0
dtype: int64

In [167]:
foursquare_resturent.to_csv('foursquare_kolkata.csv')